<img width=150 src="https://upload.wikimedia.org/wikipedia/commons/thumb/1/1a/NumPy_logo.svg/200px-NumPy_logo.svg.png"></img>

# Part.2-1-02 NumPy 陣列進階操作

範例目標:<br>
1. 實做陣列重塑
2. 實做陣列的合併與分割
3. 實做陣列的迭代
4. 實做陣列的搜尋與排序


範例重點:<br>
1. 重塑、合併、分割、迭代、搜尋、排序等操作應注意軸 (axis) 與維度 (dimension) ，常因為軸及維度、形狀等問題而造成程式錯誤


In [ ]:
import numpy as np

## 1. NumPy 陣列重塑

### 1.1 `flatten()` 與 `ravel()`

透過 `flatten()` 與 `ravel()` 均可將多維陣列轉形為一維陣列，`flatten()` 與 `ravel()` 的使用透過下列兩種方法，得到的結果都是完全一樣的。

|np.函式|陣列物件.函式|
|---|---|
|np.flatten(a, order='C')|a.flatten(order='C')|
|np.ravel(a, order='C')|a.ravel(order='C')|

In [ ]:
a = np.array([[ 0,  1,  2,  3], 
              [ 4,  5,  6,  7],
              [ 8,  9, 10, 11]])
a

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [ ]:
a.flatten()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

與 `flatten()` 不同的是，`ravel()` 建立的是原來陣列的 view，所以在 `ravel()` 回傳物件中做的元素值變更，將會影響原陣列的元素值。

In [ ]:
b = a.ravel()
b

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

如果我們改變 b 陣列的元素值，原陣列 a 對應的元素值也會被改變。

In [ ]:
b[3] = 100

In [ ]:
b

array([  0,   1,   2, 100,   4,   5,   6,   7,   8,   9,  10,  11])

In [ ]:
a

array([[  0,   1,   2, 100],
       [  4,   5,   6,   7],
       [  8,   9,  10,  11]])

`flatten()` 與 `ravel()` 引數 order 預設值為 C，常用的引數值有 C 和 F。C 的意義是 C-style，展開時是以 row 為主的順序展開；而 F 是 Fortran-style，展開時是以 column 為主的順序展開。

In [ ]:
a.ravel(order='C')

array([  0,   1,   2, 100,   4,   5,   6,   7,   8,   9,  10,  11])

In [ ]:
a.ravel(order='F')

array([  0,   4,   8,   1,   5,   9,   2,   6,  10, 100,   7,  11])

### 1.2 `reshape()`

`reshape()` 的使用，可以透過 `np.reshape(a, new_shape)` 或 `a.reshape(new_shape, refcheck=True)` 來執行。

In [ ]:
a = np.arange(15)

In [ ]:
a

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])

In [ ]:
a.reshape((3, 5))

array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14]])

如果新的總數與原先 shape 總數不一致的話，則會產生錯誤。

In [ ]:
a.size

15

In [ ]:
a.reshape((3, 6))

ValueError: cannot reshape array of size 15 into shape (3,6)

Reshape 時，新的形狀可以採用模糊指定為 -1，讓 NumPy 自動計算。

In [ ]:
a.reshape((5, -1))

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11],
       [12, 13, 14]])

若 reshape 後的陣列元素值改變，會影響原陣列對應的元素值也跟著改變。

In [ ]:
b = a.reshape((3, 5))
b[0, 2] = 100
b

array([[  0,   1, 100,   3,   4],
       [  5,   6,   7,   8,   9],
       [ 10,  11,  12,  13,  14]])

a[2] 值被改變了。

In [ ]:
a

array([  0,   1, 100,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14])

### 1.3 `resize()`

`resize()` 的使用，也同樣可以透過 `np.resize(a, new_shape)` 或 `a.resize(new_shape, refcheck=True)` 來執行。

要改變被 reference 的陣列時有可能會產生錯誤，這時候可以將 `refcheck` 引數設為 `False` (預設為 `True`)。

In [ ]:
b = np.arange(15)
b

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])

如果 resize 的大小超過總元素值，則會在後面的元素值的指定為 0。

In [ ]:
b.size

15

In [ ]:
b.resize((3, 6), refcheck=False)
b

array([[ 0,  1,  2,  3,  4,  5],
       [ 6,  7,  8,  9, 10, 11],
       [12, 13, 14,  0,  0,  0]])

如果 resize 的大小小於總元素值，則會依照 C-style 的順序，取得 resize 後的陣列元素。

In [ ]:
b.resize(3, refcheck=False)
b

array([0, 1, 2])

## 2. 軸 (axis) 與維度 (dimension)

軸 (axis) 在 NumPy 多維陣列中是很重要觀念，但是在應用上容易混淆。軸的數目也就是 NumPy 陣列的維度 (dimension) 數，軸的順序編號從 0 開始，下面例子用圖示來解說。

### 2.1 一維陣列的軸

對一維陣列來說，只有一個軸，所以 axis 為 0。

In [ ]:
a = np.array([1, 2, 3])
a

array([1, 2, 3])

以圖示來說明一維陣列的軸。

![](1D_axis.png)

### 2.2 二維陣列的軸

二維陣列的 ndim 為 2，也就是會有 2 個軸。二維陣列的軸 0 就是沿著 row 的軸，而軸 1 是沿著 column 的軸。

In [ ]:
a = np.arange(6).reshape(3, 2)
a

array([[0, 1],
       [2, 3],
       [4, 5]])

以圖示來說明二維陣列的軸。

![](2D_axis.png)

### 2.3 三維陣列的軸

三維陣列有 3 個軸。可以理解軸的順序是"由外而內"、"由row而column"。

以前一天範例程式中三維陣列的例子來看，可以理解為 2 個 4 $\times$ 3 的二維陣列排在一起。

In [ ]:
a = np.array([[[1, 2, 3], [4, 5, 6],
              [7, 8, 9], [10, 11, 12]],
              [[1, 2, 3], [4, 5, 6],
              [7, 8, 9], [10, 11, 12]]])
a

array([[[ 1,  2,  3],
        [ 4,  5,  6],
        [ 7,  8,  9],
        [10, 11, 12]],

       [[ 1,  2,  3],
        [ 4,  5,  6],
        [ 7,  8,  9],
        [10, 11, 12]]])

以圖示來說明三維陣列的軸。

![](3D_axis.png)

從 `shape` 屬性來看也可以協助理解在多維陣列中的軸。

In [ ]:
a.shape

(2, 4, 3)

若我們要沿軸對元素做加總，呼叫 `sum()` 函式並指定 axis。

In [ ]:
a.sum(axis=0)

array([[ 2,  4,  6],
       [ 8, 10, 12],
       [14, 16, 18],
       [20, 22, 24]])

### 2.4 `np.newaxis` 增加軸數

跟 `reshape()` 類似的應用，如果是要增加軸數的話，可以使用 `np.newaxis` 物件。將 `np.newaxis` 加到要增加的軸的位置即可。

與 `reshape()` 不同的地方在於，`np.newaxis` 新增的維度為 1，而 `reshape()` 可以指定要改變的形狀 (不一定為 1)。

In [ ]:
a = np.arange(12).reshape(2, 6)
a

array([[ 0,  1,  2,  3,  4,  5],
       [ 6,  7,  8,  9, 10, 11]])

In [ ]:
a[:,np.newaxis,:].shape

(2, 1, 6)

## 3. NumPy 陣列的合併與分割

In [ ]:
a = np.arange(10).reshape(5, 2)
b = np.arange(6).reshape(3, 2)

In [ ]:
a

array([[0, 1],
       [2, 3],
       [4, 5],
       [6, 7],
       [8, 9]])

In [ ]:
b

array([[0, 1],
       [2, 3],
       [4, 5]])

### 3.1 合併：`concatenate()`, `stack()`, `hstack()`, `vstack()`

使用 `concatenate()` 進行陣列的合併時，須留意除了指定的軸之外 (預設為 axis 0)，其他軸的形狀必須完全相同，合併才不會發生錯誤。

```python
numpy.concatenate((a1, a2, ...), axis=0, out=None)
```

In [ ]:
np.concatenate((a, b))

array([[0, 1],
       [2, 3],
       [4, 5],
       [6, 7],
       [8, 9],
       [0, 1],
       [2, 3],
       [4, 5]])

`stack()`, `hstack()`, `vstack()` 的觀念及用法類似，不同點在於 `stack()` 回傳的陣列維度會是合併前的維度 +1，而 `hstack()` 與 `vstack()` 回傳的陣列維度則是依合併的陣列而定。

至於是否可以合併，`stack()` 必須要所有陣列的形狀都一樣；而 `hstack()` 與 `vstack()` 則跟上述的規則一樣，除了指定的軸之外，其他軸的形狀必須完全相同才可以合併。

|函式|說明|
|---|---|
|numpy.stack(arrays, axis=0, out=None)|根據指定的軸進行合併|
|numpy.hstack(tup)|根據水平軸進行合併|
|numpy.vstack(tup)|根據垂直軸進行合併|

In [ ]:
# stack() 範例
c = np.arange(10).reshape(5, 2)
np.stack((a, c), axis=1)

array([[[0, 1],
        [0, 1]],

       [[2, 3],
        [2, 3]],

       [[4, 5],
        [4, 5]],

       [[6, 7],
        [6, 7]],

       [[8, 9],
        [8, 9]]])

In [ ]:
# hstack() 範例
np.hstack((a, c))

array([[0, 1, 0, 1],
       [2, 3, 2, 3],
       [4, 5, 4, 5],
       [6, 7, 6, 7],
       [8, 9, 8, 9]])

In [ ]:
# vstack() 範例
np.vstack((a, b))

array([[0, 1],
       [2, 3],
       [4, 5],
       [6, 7],
       [8, 9],
       [0, 1],
       [2, 3],
       [4, 5]])

### 3.2 分割：`split()`、`hsplit()`、`vsplit()`

In [ ]:
a = np.arange(10).reshape(5, 2)
a

array([[0, 1],
       [2, 3],
       [4, 5],
       [6, 7],
       [8, 9]])

`split()` 的語法：

```python
numpy.split(array, indices_or_sections, axis=0)
```

indices_or_sections 如果給定單一整數的話，那就會按照軸把陣列等分；如果給定一個 List 的整數值的話，就會按照區段去分割，例如：
`indices_or_sections=[2, 3]` 會照下列的方式做分割 (一樣是照按照軸把陣列分割)
```
ary[:2]
ary[2:3]
ary[3:]
```

與 `split` 很類似的是 `hsplit` 與 `vsplit`，分別是依照水平軸和垂直軸去做分割。

In [ ]:
# 依 axis 0 等分 split
np.split(a, 5)

[array([[0, 1]]),
 array([[2, 3]]),
 array([[4, 5]]),
 array([[6, 7]]),
 array([[8, 9]])]

In [ ]:
# split 為 (2,2), (1,2), (2,2) 三個陣列，並回傳含 3 個陣列的 List
np.split(a, [2,3])

[array([[0, 1],
        [2, 3]]), array([[4, 5]]), array([[6, 7],
        [8, 9]])]

In [ ]:
b = np.arange(30).reshape(5, 6)
b

array([[ 0,  1,  2,  3,  4,  5],
       [ 6,  7,  8,  9, 10, 11],
       [12, 13, 14, 15, 16, 17],
       [18, 19, 20, 21, 22, 23],
       [24, 25, 26, 27, 28, 29]])

In [ ]:
# 依水平軸去做等分分割
np.hsplit(b, 3)

[array([[ 0,  1],
        [ 6,  7],
        [12, 13],
        [18, 19],
        [24, 25]]), array([[ 2,  3],
        [ 8,  9],
        [14, 15],
        [20, 21],
        [26, 27]]), array([[ 4,  5],
        [10, 11],
        [16, 17],
        [22, 23],
        [28, 29]])]

In [ ]:
# 依水平軸照區段去分割
np.hsplit(b, [2, 3, 5])

[array([[ 0,  1],
        [ 6,  7],
        [12, 13],
        [18, 19],
        [24, 25]]), array([[ 2],
        [ 8],
        [14],
        [20],
        [26]]), array([[ 3,  4],
        [ 9, 10],
        [15, 16],
        [21, 22],
        [27, 28]]), array([[ 5],
        [11],
        [17],
        [23],
        [29]])]

In [ ]:
# 依垂直軸按照區段去分割，超出的區段則傳回空陣列
np.vsplit(b, [2, 4, 6])

[array([[ 0,  1,  2,  3,  4,  5],
        [ 6,  7,  8,  9, 10, 11]]), array([[12, 13, 14, 15, 16, 17],
        [18, 19, 20, 21, 22, 23]]), array([[24, 25, 26, 27, 28, 29]]), array([], shape=(0, 6), dtype=int64)]

## 4. 迭代

一維陣列的迭代，跟 Python 集合型別 (例如 List) 的迭代相同。

In [ ]:
a = np.arange(5)
a

array([0, 1, 2, 3, 4])

In [ ]:
for i in a:
    print(i)

0
1
2
3
4


多維陣列的迭代則以 axis 0 為準。下面以二維陣列為例，列出各 row 的元素。

In [ ]:
b = np.arange(6).reshape(2, 3)
b

array([[0, 1, 2],
       [3, 4, 5]])

In [ ]:
for row in b:
    print(row)

[0 1 2]
[3 4 5]


如果要列出多維陣列所有元素的話，可以配合 `flat` 屬性。

In [ ]:
for i in b.flat:
    print(i)

0
1
2
3
4
5


## 5. 搜尋與排序

### 5.1 顯示最大值和最小值：`amax()`、`amin()`、`max()`、`min()`

要顯示陣列元素最大值和最小值，可以透過 `amax()`、`amin()`、`max()`、`min()`，也可以依照軸列出各軸的最大/最小元素值。

基本語法：

|np.函式|陣列物件.函式|
|---|---|
|numpy.amax(array, axis=None, keepdims=<no value>)|ndarray.max(axis=None, keepdims=False)|
|numpy.amin(array, axis=None, keepdims=<no value>)|ndarray.min(axis=None, keepdims=False)|

In [ ]:
a = np.random.randint(1, 20, 10)
a

array([12, 12, 11,  9,  6, 14,  5, 10,  3, 15])

In [ ]:
# 陣列中最大的元素值
np.amax(a)

15

In [ ]:
# 陣列中最小的元素值
np.amin(a)

3

如果是多維陣列的話，用法也是相同，也可以依照軸列出最大或最小值。

In [ ]:
b = a.reshape(2, 5)
b

array([[12, 12, 11,  9,  6],
       [14,  5, 10,  3, 15]])

In [ ]:
# 若設定 keepdims=True，結果會保留原陣列的維度來顯示。
np.amax(b, keepdims=True)

array([[15]])

In [ ]:
# 列出各 row 最大值
b.max(axis=1)

array([12, 15])

In [ ]:
# 同樣的 amax 也可以依軸列出各 row 最大值
np.amax(b, axis=1)

array([12, 15])

In [ ]:
# 列出各 column 最小值
b.min(axis=0)

array([12,  5, 10,  3,  6])

### 5.2 顯示最大值和最小值的索引：`argmax()` 與 `argmin()`

`argmax` / `argmin` 和上述不同的地方在於，`argmax` / `argmin` 回傳的是最大值和最小值的索引，也可以依照軸找出各軸最大值和最小值的索引。

基本語法：

|np.函式|陣列物件.函式|
|---|---|
|numpy.argmax(array, axis=None)|ndarray.argmax(axis=None)|
|numpy.argmin(array, axis=None)|ndarray.argmin(axis=None)|

In [ ]:
np.random.seed(0)
a = np.random.randint(1, 20, size=(3, 4))
a

array([[13, 16,  1,  4],
       [ 4,  8, 10, 19],
       [ 5,  7, 13,  2]])

若沒有指定軸的話，`argmax()` 與 `argmin()` 會回傳多維陣列展平後的索引。

In [ ]:
np.argmax(a)

7

In [ ]:
# 列出各 column 的最大值索引, 分別為 [0, 0, 2, 1]
np.argmax(a, axis=0)

array([0, 0, 2, 1])

In [ ]:
# 元素值 1 為最小值，展平後的索引值為 2。
a.argmin()

2

### 5.3 找出符合條件的元素：`where`

語法：
```python
numpy.where(condition[, x, y])
```

In [ ]:
a

array([[13, 16,  1,  4],
       [ 4,  8, 10, 19],
       [ 5,  7, 13,  2]])

傳入條件式，回傳值為符合條件的元素索引，不過這邊留意的是，以下面二維陣列為例，回傳的索引陣列要合併一起看，也就是說
```
(array([0, 0, 1, 2]), 
 array([0, 1, 3, 2]))
```

a[0, 0] 值為 13<br />
a[0, 1] 值為 16<br />
a[1, 3] 值為 19<br />
a[2, 2] 值為 13

以上索引值對應的元素，其值都符合 "大於 10" 的條件。

In [ ]:
np.where(a > 10)

(array([0, 0, 1, 2]), array([0, 1, 3, 2]))

若是設定 x, y 引數的話，可將各元素取代掉。以下面的例子來解釋，如果元素值大於 10 的話就用 "Y" 來替代，反之則是 "N"。

In [ ]:
np.where(a > 10, "Y", "N")

array([['Y', 'Y', 'N', 'N'],
       ['N', 'N', 'N', 'Y'],
       ['N', 'N', 'Y', 'N']], dtype='<U1')

### 5.4 `nonzero`

`nonzero` 等同於 `np.where(array != 0)` 的語法，同樣的也是回傳符合非 0 條件的元素索引值。

語法：

|np.函式|陣列物件.函式|
|---|---|
|numpy.nonzero(array)|ndarray.nonzero()|

In [ ]:
np.random.seed(2)
a = np.random.randint(0, 5, 10)
a

array([0, 0, 3, 2, 3, 0, 2, 1, 3, 2])

In [ ]:
np.nonzero(a)

(array([2, 3, 4, 6, 7, 8, 9]),)

In [ ]:
a.nonzero()

(array([2, 3, 4, 6, 7, 8, 9]),)

### 5.5 排序：`sort()` 與 `argsort()`

要對陣列進行排序可以使用 `sort()` 與 `argsort()`，兩者的差異是在 `sort()` 回傳的是排序後的陣列，而 `argsort()` 回傳的是排序後的陣列索引值。

語法：

|np.函式|陣列物件.函式|
|---|---|
|numpy.sort(a, axis=-1, kind=None, order=None)|ndarray.sort()|
|numpy.argsort(a, axis=-1, kind=None, order=None)|ndarray.argsort()|

In [ ]:
np.random.seed(3)
a = np.random.randint(0, 20, 10)
a

array([10,  3,  8,  0, 19, 10, 11,  9, 10,  6])

In [ ]:
np.sort(a)

array([ 0,  3,  6,  8,  9, 10, 10, 10, 11, 19])

In [ ]:
a.argsort()

array([3, 1, 9, 2, 7, 0, 5, 8, 6, 4])

與 `np.sort()` 不同的是，`陣列物件.sort()` 的語法會進行 in-place 排序，也就是原本的陣列內容會跟著改變。

In [ ]:
a.sort()
a

array([ 0,  3,  6,  8,  9, 10, 10, 10, 11, 19])

多維陣列在排序時可以指定要依據的軸。

In [ ]:
b = np.random.randint(0, 20, size=(5, 4))
b

array([[ 0, 12,  7, 14],
       [17,  2,  2,  1],
       [19,  5,  8, 14],
       [ 1, 10,  7, 11],
       [ 1, 15, 16,  5]])

In [ ]:
np.sort(b, axis=0)

array([[ 0,  2,  2,  1],
       [ 1,  5,  7,  5],
       [ 1, 10,  7, 11],
       [17, 12,  8, 14],
       [19, 15, 16, 14]])

排序支援多種不同的排序算法，包括 quicksort (預設)、heapsort、mergesort、timesort，在 `kind` 引數指定即可。依照官網文件指出排序速度是以 quicksort 最快，mergesort / timesort 其次，之後是 heapsort。

In [ ]:
c = np.random.randint(0, 100000000, 1000000)
np.sort(c, kind='heapsort')

array([      64,       96,      310, ..., 99999479, 99999561, 99999830])